In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import Adam
import joblib
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedBaggingClassifier

# 결과 저장 경로 생성
output_dir = "../results/models/"
os.makedirs(output_dir, exist_ok=True)

# 데이터 로드
df_cleaned = pd.read_csv("../data/processed/data_cleaned.csv")

# 데이터 분할 (특성(X)과 레이블(y) 분리)
X = df_cleaned.drop(columns=['Label'])
y = df_cleaned['Label']

# 훈련/테스트 데이터 분할 (80% 훈련, 20% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 데이터 정규화 (평균 0, 표준편차 1로 변환)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [4]:
# Autoencoder 커널 정의
input_dim = X_train_scaled.shape[1]  # 입력 특성의 차원 (63)
hidden_dim1 = 56  # 첫 번째 은닉층 차원
encoding_dim = 49  # 차원 축소 목표 차원

# 입력층 정의
input_layer = Input(shape=(input_dim,))

# 인코더 네트워크 (차원 축소)
encoded = Dense(hidden_dim1, activation='relu')(input_layer)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# 디코더 네트워크 (복원 과정)
decoded = Dense(hidden_dim1, activation='relu')(encoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# 분류기 출력 (이진 분류기 추가)
classification_output = Dense(1, activation='sigmoid')(encoded)

# 1. Autoencoder + Binary Classifier 모델 정의
autoencoder_classifier = Model(inputs=input_layer, outputs=[decoded, classification_output])
autoencoder_classifier.compile(
    optimizer=Adam(),
    loss=['mean_squared_error', 'binary_crossentropy'],  # Autoencoder와 분류 손실
    loss_weights=[0.5, 0.5],  # 손실 가중치
    metrics={'dense_4': 'accuracy'}  # 분류기 정확도 추적
)

# EarlyStopping 콜백 설정 (모델이 더 이상 개선되지 않으면 학습 중단)
early_stopping = EarlyStopping(
    monitor='val_dense_4_accuracy',  # 정확도 모니터링
    patience=5,
    restore_best_weights=True,
    mode='max'  # 정확도를 최대화하려면 'max'
)

# Autoencoder + Binary Classifier 모델 학습
y_binary_train = (y_train != 0).astype(int)  # 이진 분류 (0이면 0, 아니면 1)
y_binary_test = (y_test != 0).astype(int)

autoencoder_classifier.fit(
    X_train_scaled, [X_train_scaled, y_binary_train],  # 입력과 출력(원본 데이터와 분류 레이블)
    epochs=50,
    batch_size=128,
    validation_data=(X_test_scaled, [X_test_scaled, y_binary_test]),
    callbacks=[early_stopping]
)

# 모델 저장 (Keras 형식으로 저장)
autoencoder_classifier.save(os.path.join(output_dir, "autoencoder_classifier.keras"))


MemoryError: Unable to allocate 544. MiB for an array with shape (2264474, 63) and data type float32

In [ ]:
# Autoencoder 차원 축소 (인코더만 추출)
encoder = Model(inputs=input_layer, outputs=encoded)

# 차원 축소된 데이터 생성
X_train_encoded = encoder.predict(X_train_scaled)
X_test_encoded = encoder.predict(X_test_scaled)

# 차원 축소된 데이터 차원 확인
print("Original shape:", X_train_scaled.shape)  # 원래 차원: (샘플 수, 63)
print("Reduced shape:", X_train_encoded.shape)  # 축소된 차원: (샘플 수, 49)

In [ ]:
# 2. Random Forest 이진 분류 모델 학습 (차원 축소된 데이터 사용)
rf = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10, n_jobs=-1)
rf.fit(X_train_encoded, y_binary_train)

# Random Forest 모델 저장
joblib.dump(rf, os.path.join(output_dir, "autoencoder_random_forest.pkl"))

# 인코더 모델 저장 (HDF5 대신 Keras 형식으로 저장)
encoder.save(os.path.join(output_dir, "encoder.keras"))

In [ ]:
# 3. UDBB 설정 및 Random Forest 기반 모델 정의

# UDBB 설정 및 Random Forest 기반 모델 정의
base_estimator = RandomForestClassifier(
    n_estimators=100, random_state=42, n_jobs=-1
)

udbb_model = BalancedBaggingClassifier(
    estimator=base_estimator,  # base_estimator -> estimator로 변경
    sampling_strategy='auto',   # 자동으로 언더샘플링 비율 설정
    replacement=False,          # 샘플링 시 중복 허용하지 않음
    random_state=42,
    n_estimators=100,           # 앙상블에 포함할 모델 개수
    n_jobs=-1                   # 병렬 처리
)

# 모델 훈련 (차원 축소된 데이터를 사용)
udbb_model.fit(X_train_encoded, y_train)

# 모델 저장
joblib.dump(udbb_model, os.path.join(output_dir, "autoencoder_udbb.pkl"))

In [ ]:
# 4. CNN 분류기 모델 구성 (Autoencoder로 차원 축소된 데이터 사용)
cnn_classifier = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(encoding_dim, 1)),  # 인코딩된 데이터 차원 사용
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(15, activation='softmax')  # 다중 클래스 분류를 위한 소프트맥스 출력
])

# 모델 컴파일 (Adam optimizer와 sparse_categorical_crossentropy 손실 함수 사용)
cnn_classifier.compile(
    optimizer='adam',  # 최적화 기법
    loss='sparse_categorical_crossentropy',  # 다중 클래스 분류 손실 함수
    metrics=['accuracy']  # 평가 지표로 정확도 사용
)

# CNN 학습을 위해 입력 데이터 차원 변경 (CNN은 3D 입력을 요구)
X_train_encoded_cnn = np.expand_dims(X_train_encoded, axis=-1)  # (샘플 수, 차원 수, 1)
X_test_encoded_cnn = np.expand_dims(X_test_encoded, axis=-1)    # (샘플 수, 차원 수, 1)

# CNN 모델 학습
cnn_classifier.fit(
    X_train_encoded_cnn, y_train,  # 타겟 데이터는 정수형(0~14)
    epochs=30,
    batch_size=64,
    validation_data=(X_test_encoded_cnn, y_test)
)

# CNN 모델 저장
cnn_classifier.save(os.path.join(output_dir, "autoencoder_cnn.keras"))